In [1]:
%matplotlib inline
import pystac_client
import numpy as np
from matplotlib import pyplot as plt
import geopandas as gpd

# Search the CMR STAC API

This notebook uses the CMR STAC API to search for scenes with a bounding box.
It's trivial to add a datetime range and other parameters to find scenes and
plausible to merge multiple resulting datasets into a temporal stack.

In [7]:
client = pystac_client.Client.open("https://cmr.earthdata.nasa.gov/cloudstac/LPCLOUD")

R = 0.1
pt = (147, -36)
bbox = [pt[0]-R, pt[1]-R, pt[0] +R, pt[1] + R]
#bbox=[149.0, -35.5, 149.5, -35.0]

# Search with a bbox over Canberra, Australia
results = client.search(
    collections=["EMITL2ARFL.v001"], 
    #datetime="2023-01-01/2023-10-31",
    bbox = bbox,
)

print(f"Found {results.matched()} items")
print("... fetching metadata")
items = list(results.items())
print("... done")

Found 3 items
... fetching metadata
... done


In [13]:
from IPython.display import Image

Image(url=items[0].assets['browse'].href)

In [19]:
item = items[0]

uu = []

for asset in item.assets.values():
    s3_url = asset.href.replace("https://data.lpdaac.earthdatacloud.nasa.gov/", "s3://")
    if s3_url.startswith("s3://") and s3_url.endswith(".nc"):
        uu.append(s3_url)
print("\n".join(uu))

s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230316T045133_2307503_005/EMIT_L2A_RFL_001_20230316T045133_2307503_005.nc
s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230316T045133_2307503_005/EMIT_L2A_RFLUNCERT_001_20230316T045133_2307503_005.nc
s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20230316T045133_2307503_005/EMIT_L2A_MASK_001_20230316T045133_2307503_005.nc


In [4]:
def get_orbit(item):
    if isinstance(item, str):
        _id = item
    else:
        _id = item.id
        
    return int(_id.split('_')[-2])

for item in items:
    item.properties['orbit'] = get_orbit(item)
    item.properties['label'] = item.id

In [6]:
gdf = gpd.GeoDataFrame.from_features(items, "epsg:4326")

#display(gdf.head())

#_gdf = gdf[gdf.orbit == 2329804]

gdf.explore(
    tooltip=[
        "orbit",
        "datetime",
        "label",
        "eo:cloud_cover",
    ],
)

----------------------------------------------

In [ ]:
# Albury, Lake Hume